In [1]:
import torch
from supervoice_gpt import SupervoiceGPT, Tokenizer, config

In [2]:
# Model
device = "cpu"
tokenizer = Tokenizer(config, "tokenizer_text.model")
model = SupervoiceGPT(config).to(device)
checkpoint = torch.load(f'./output/big_dataset.pt', map_location=device)
model.load_state_dict(checkpoint['model'])
model.eval()
print(checkpoint['step'])

232000


In [12]:
tokens = model.generate("What do you want to do today? I want to play football with my friends.", tokenizer, max_new_tokens = 256, top_k = 6, device = device)
print(tokens)
print(len(tokens))

[('w', 5, 255), ('ɐ', 5, 255), ('ʔ', 4, 255), ('<SIL>', 0, 255), ('d', 2, 255), ('ʉː', 3, 255), ('<SIL>', 0, 255), ('j', 3, 255), ('ʉː', 4, 255), ('<SIL>', 0, 255), ('w', 4, 255), ('ɒ', 8, 252), ('n', 4, 231), ('<SIL>', 0, 192), ('tʰ', 6, 176), ('ʊ', 3, 171), ('<SIL>', 0, 170), ('d', 4, 168), ('ʉː', 8, 161), ('<SIL>', 0, 157), ('tʰ', 8, 151), ('ə', 2, 143), ('d', 5, 138), ('ej', 27, 112), ('<SIL>', 26, 85), ('aj', 10, 86), ('<SIL>', 0, 87), ('w', 7, 88), ('ɒ', 6, 88), ('n', 5, 87), ('t', 0, 88), ('<SIL>', 0, 88), ('t', 3, 88), ('ə', 4, 90), ('<SIL>', 0, 91), ('p', 7, 92), ('l', 3, 94), ('ej', 6, 96), ('<SIL>', 0, 98), ('f', 11, 101), ('ʊ', 6, 106), ('t', 8, 109), ('b', 4, 113), ('ɒ', 3, 117), ('ɫ', 7, 121), ('<SIL>', 0, 125), ('w', 6, 125), ('ɪ', 3, 114), ('θ', 7, 104), ('<SIL>', 0, 102), ('m', 3, 102), ('aj', 9, 100), ('<SIL>', 0, 98), ('f', 9, 95), ('ɹ', 4, 93), ('ɛ', 10, 88), ('n', 8, 88), ('z', 17, 86)]
58


In [4]:
# input = "Hey, you?"
# ctx_tokens = ['ç', 'ɪ']
# ctx_durations = [8, 10]
# probs, p = model.predict_next(input, ctx_tokens, ctx_durations, tokenizer)
# for i in range(len(probs)):
#     print(p[i] + ": "+ str(probs[i]))
